In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import optuna
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 사용 시
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [2]:
# 1. 데이터 로딩
df = pd.read_csv('데이터프레임_수정_v05.csv', encoding = 'cp949') 

# 단속건수 기준 내림차순 정렬
df_sorted = df.sort_values(by='단속건수', ascending=False)

In [3]:
df['적정 주차면수'] = df['주차면수 합'] + df['가중치*단속건수']

# 2. 독립변수 / 종속변수 지정
X = df[['생활 인구', '소매업', '의료기관', '교육기관', '카드소비', '서비스업', '음식점', '지역면적(km^2)', '버스정류장수', '추정 차량등록수', '근린생활시설수']]
y = df['적정 주차면수']  # = 주차면수 합 + 단속건수*가중치

# 모델별 변수 선택 및 성능 비교 전략  
본 프로젝트에서는 XGBoost, Random Forest, CatBoost 세 가지 모델의 성능을 공정하게 비교하기 위해, 각 모델에 대해 Optuna 기반 하이퍼파라미터 튜닝과 변수 선택(feature selection)을 동시에 수행하였다.

이를 위해 다음과 같은 전략을 사용하였다:

각 변수에 대해 사용 여부(use or not)를 이진 변수로 설정하고,

Optuna가 이를 포함한 채 동시에 하이퍼파라미터 최적값을 탐색하도록 구성하였다.

즉, 모델이 스스로 “어떤 변수를 포함할지”와 “어떻게 튜닝할지”를 함께 학습한다.

이 과정을 통해 얻은 각 모델의  
- 최적 하이퍼파라미터 조합
- 선택된 변수 목록
- 교차검증 기반 성능 지표(R² 등)

을 비교하여 최종 사용할 모델을 선정한다.

In [37]:
def objective_xgb(trial):
    # 리스트 내포로 변수 선택
    selected_features = [col for col in X.columns if trial.suggest_categorical(f"use_{col}", [True, False])]

    # 최소 1개 변수 선택 강제
    if len(selected_features) == 0:
        return -float("inf")

    X_selected = X[selected_features]

    # 하이퍼파라미터 튜닝
    params = {
        'n_estimators': 100,
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = XGBRegressor(**params, random_state=0)
    score = cross_val_score(model, X_selected, y, cv=10, scoring='r2').mean()
    return score

# Optuna 실행
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)

# 결과 출력
print("🎯 Best R²:", study_xgb.best_value)
print("✅ Best params:", study_xgb.best_params)

# 선택된 변수 목록 추출
selected_cols = [col for col in X.columns if study_xgb.best_params.get(f"use_{col}", False)]
print("🔍 선택된 변수들:", selected_cols)

[I 2025-05-20 14:31:44,067] A new study created in memory with name: no-name-29965af6-b07e-40c3-a3bb-bee34114783f
[I 2025-05-20 14:31:44,426] Trial 0 finished with value: 0.23847232203707494 and parameters: {'use_생활 인구': False, 'use_소매업': False, 'use_의료기관': True, 'use_교육기관': True, 'use_카드소비': True, 'use_서비스업': True, 'use_음식점': False, 'use_지역면적(km^2)': False, 'use_버스정류장수': False, 'use_추정 차량등록수': False, 'use_근린생활시설수': False, 'max_depth': 6, 'learning_rate': 0.0570378819241197, 'subsample': 0.90496500967159, 'colsample_bytree': 0.6936090445217861, 'reg_alpha': 0.4526461051639019, 'reg_lambda': 1.3449286615176914, 'min_child_weight': 9}. Best is trial 0 with value: 0.23847232203707494.
[I 2025-05-20 14:31:44,947] Trial 1 finished with value: 0.36701292678324476 and parameters: {'use_생활 인구': False, 'use_소매업': False, 'use_의료기관': False, 'use_교육기관': True, 'use_카드소비': False, 'use_서비스업': True, 'use_음식점': False, 'use_지역면적(km^2)': False, 'use_버스정류장수': True, 'use_추정 차량등록수': True, 'use_근린생활시설수': Tru

🎯 Best R²: 0.6266675932447636
✅ Best params: {'use_생활 인구': False, 'use_소매업': True, 'use_의료기관': True, 'use_교육기관': False, 'use_카드소비': False, 'use_서비스업': True, 'use_음식점': True, 'use_지역면적(km^2)': False, 'use_버스정류장수': True, 'use_추정 차량등록수': True, 'use_근린생활시설수': False, 'max_depth': 4, 'learning_rate': 0.06065590530061637, 'subsample': 0.7298546057127474, 'colsample_bytree': 0.7040705513767808, 'reg_alpha': 2.2310999733982517, 'reg_lambda': 3.972303696991541, 'min_child_weight': 2}
🔍 선택된 변수들: ['소매업', '의료기관', '서비스업', '음식점', '버스정류장수', '추정 차량등록수']


In [15]:
def objective_cat(trial):
    # 변수 선택
    selected_features = [col for col in X.columns if trial.suggest_categorical(f"use_{col}", [True, False])]
    if len(selected_features) == 0:
        return -float("inf")
    X_selected = X[selected_features]

    # 하이퍼파라미터
    params = {
        'iterations': trial.suggest_int('iterations', 100, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0)
    }

    model = CatBoostRegressor(**params, verbose=0, random_state=0)
    score = cross_val_score(model, X_selected, y, cv=10, scoring='r2').mean()
    return score

study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=50)

print("🎯 [CatBoost] Best R²:", study_cat.best_value)
print("✅ Best params:", study_cat.best_params)
selected_cols_cat = [col for col in X.columns if study_cat.best_params.get(f"use_{col}", False)]
print("🔍 선택된 변수들:", selected_cols_cat)

[I 2025-05-20 14:00:47,913] A new study created in memory with name: no-name-c02ae9a3-b46a-4221-8b9b-0c19dbf56c00
[I 2025-05-20 14:00:48,938] Trial 0 finished with value: 0.3765526657710846 and parameters: {'use_생활 인구': True, 'use_소매업': False, 'use_의료기관': False, 'use_교육기관': False, 'use_카드소비': True, 'use_서비스업': False, 'use_음식점': False, 'use_지역면적(km^2)': False, 'use_버스정류장수': False, 'use_추정 차량등록수': False, 'use_근린생활시설수': True, 'iterations': 129, 'depth': 3, 'learning_rate': 0.0583522496270607, 'l2_leaf_reg': 2.501035398285321, 'bagging_temperature': 0.05805533431418819, 'random_strength': 0.04541565124488589}. Best is trial 0 with value: 0.3765526657710846.
[I 2025-05-20 14:00:50,497] Trial 1 finished with value: 0.017259494536950436 and parameters: {'use_생활 인구': False, 'use_소매업': False, 'use_의료기관': False, 'use_교육기관': False, 'use_카드소비': True, 'use_서비스업': False, 'use_음식점': False, 'use_지역면적(km^2)': True, 'use_버스정류장수': True, 'use_추정 차량등록수': True, 'use_근린생활시설수': False, 'iterations': 121, 'dept

🎯 [CatBoost] Best R²: 0.6001463913416216
✅ Best params: {'use_생활 인구': False, 'use_소매업': True, 'use_의료기관': False, 'use_교육기관': False, 'use_카드소비': True, 'use_서비스업': False, 'use_음식점': True, 'use_지역면적(km^2)': False, 'use_버스정류장수': True, 'use_추정 차량등록수': False, 'use_근린생활시설수': False, 'iterations': 222, 'depth': 3, 'learning_rate': 0.08468615081647429, 'l2_leaf_reg': 7.4645110045971395, 'bagging_temperature': 0.9086172715884305, 'random_strength': 0.0672721307121702}
🔍 선택된 변수들: ['소매업', '카드소비', '음식점', '버스정류장수']


In [18]:
def objective_rf(trial):
    # 변수 선택
    selected_features = [col for col in X.columns if trial.suggest_categorical(f"use_{col}", [True, False])]
    if len(selected_features) == 0:
        return -float("inf")
    X_selected = X[selected_features]

    # 하이퍼파라미터
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    model = RandomForestRegressor(**params, random_state=0)
    score = cross_val_score(model, X_selected, y, cv=10, scoring='r2').mean()
    return score

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)

print("🎯 [RandomForest] Best R²:", study_rf.best_value)
print("✅ Best params:", study_rf.best_params)
selected_cols_rf = [col for col in X.columns if study_rf.best_params.get(f"use_{col}", False)]
print("🔍 선택된 변수들:", selected_cols_rf)

[I 2025-05-20 14:05:06,249] A new study created in memory with name: no-name-f38ec5b6-73d3-4693-8198-fa5b1e15feb1
[I 2025-05-20 14:05:07,031] Trial 0 finished with value: 0.511543664908998 and parameters: {'use_생활 인구': True, 'use_소매업': True, 'use_의료기관': True, 'use_교육기관': True, 'use_카드소비': True, 'use_서비스업': False, 'use_음식점': True, 'use_지역면적(km^2)': False, 'use_버스정류장수': False, 'use_추정 차량등록수': True, 'use_근린생활시설수': True, 'n_estimators': 182, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.511543664908998.
[I 2025-05-20 14:05:08,733] Trial 1 finished with value: 0.43692016478766416 and parameters: {'use_생활 인구': True, 'use_소매업': True, 'use_의료기관': False, 'use_교육기관': True, 'use_카드소비': False, 'use_서비스업': True, 'use_음식점': True, 'use_지역면적(km^2)': True, 'use_버스정류장수': True, 'use_추정 차량등록수': True, 'use_근린생활시설수': True, 'n_estimators': 286, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_feat

🎯 [RandomForest] Best R²: 0.5794976883444318
✅ Best params: {'use_생활 인구': False, 'use_소매업': False, 'use_의료기관': False, 'use_교육기관': True, 'use_카드소비': True, 'use_서비스업': False, 'use_음식점': True, 'use_지역면적(km^2)': True, 'use_버스정류장수': False, 'use_추정 차량등록수': True, 'use_근린생활시설수': True, 'n_estimators': 229, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': True}
🔍 선택된 변수들: ['교육기관', '카드소비', '음식점', '지역면적(km^2)', '추정 차량등록수', '근린생활시설수']


In [41]:
def evaluate_model(study, model_class, model_name):
    # 선택된 feature
    selected_cols = [col for col in X.columns if study.best_params.get(f"use_{col}", False)]
    X_selected = X[selected_cols]
    
    n, p = X_selected.shape

    # 모델 파라미터만 추출 (use_로 시작하는 건 제외)
    best_params = {k: v for k, v in study.best_params.items() if not k.startswith('use_')}

    # 모델 인스턴스화
    if model_name == 'CatBoost':
        model = model_class(**best_params, verbose=0, random_state=0)
    else:
        model = model_class(**best_params, random_state=0)

    # R²
    r2 = cross_val_score(model, X_selected, y, cv=10, scoring='r2').mean()

    # Adjusted R²
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

    # RMSE
    neg_mse = cross_val_score(model, X_selected, y, cv=10, scoring='neg_mean_squared_error')
    rmse = np.mean(np.sqrt(-neg_mse))

    # MAE
    neg_mae = cross_val_score(model, X_selected, y, cv=10, scoring='neg_mean_absolute_error')
    mae = np.mean(-neg_mae)

    # 결과 출력
    print(f"\n📌 [{model_name}] 최적 파라미터:")
    print(best_params)
    print("🔍 선택된 변수들:", selected_cols)
    print("📊 교차검증 기반 성능 지표:")
    print(f"R²            : {r2:.5f}")
    print(f"Adjusted R²   : {adj_r2:.5f}")
    print(f"RMSE          : {rmse:.5f}")
    print(f"MAE           : {mae:.5f}")

# 세 모델 각각 평가
evaluate_model(study_xgb, XGBRegressor, "XGBoost")
evaluate_model(study_cat, CatBoostRegressor, "CatBoost")
evaluate_model(study_rf, RandomForestRegressor, "Random Forest")


📌 [XGBoost] 최적 파라미터:
{'max_depth': 4, 'learning_rate': 0.06065590530061637, 'subsample': 0.7298546057127474, 'colsample_bytree': 0.7040705513767808, 'reg_alpha': 2.2310999733982517, 'reg_lambda': 3.972303696991541, 'min_child_weight': 2}
🔍 선택된 변수들: ['소매업', '의료기관', '서비스업', '음식점', '버스정류장수', '추정 차량등록수']
📊 교차검증 기반 성능 지표:
R²            : 0.62667
Adjusted R²   : 0.56613
RMSE          : 1787.03743
MAE           : 1274.97578

📌 [CatBoost] 최적 파라미터:
{'iterations': 222, 'depth': 3, 'learning_rate': 0.08468615081647429, 'l2_leaf_reg': 7.4645110045971395, 'bagging_temperature': 0.9086172715884305, 'random_strength': 0.0672721307121702}
🔍 선택된 변수들: ['소매업', '카드소비', '음식점', '버스정류장수']
📊 교차검증 기반 성능 지표:
R²            : 0.60015
Adjusted R²   : 0.55914
RMSE          : 1822.55570
MAE           : 1301.46925

📌 [Random Forest] 최적 파라미터:
{'n_estimators': 229, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': True}
🔍 선택된 변수들: ['교육기관', '카드소비', '음식점', '지역면적(km^2)',

In [38]:
# 1. 최적 변수 선택
selected_cols = [col for col in X.columns if study_xgb.best_params.get(f"use_{col}", False)]
X_selected = X[selected_cols]

# 2. 최적 파라미터만 추출
best_params = {k: v for k, v in study_xgb.best_params.items() if not k.startswith('use_')}

# 3. 최적 모델 학습
best_model = XGBRegressor(**best_params, random_state=0)
best_model.fit(X_selected, y)  # 전체 데이터로 학습

# 4. 예측 수행
df['예측_적정주차면수'] = best_model.predict(X_selected)

# 5. 필요 주차면수 계산
df['필요주차면수'] = df['예측_적정주차면수'] - df['주차면수 합']

# 6. 우선순위지수 계산
df['우선순위지수'] = df['필요주차면수'] * (df['유입인구'] / df['유입인구'].sum())
df['우선순위지수'] = (df['우선순위지수'] / df['우선순위지수'].max()).round(3)

# 7. 우선순위 상위 10개 출력
df_sorted = df.sort_values(by='우선순위지수', ascending=False)
print("\n🏆 우선 설치 대상 상위 10개 행정동:")
print(df_sorted[['행정동', '필요주차면수', '우선순위지수']].head(10))


🏆 우선 설치 대상 상위 10개 행정동:
     행정동       필요주차면수  우선순위지수
25   인계동  5839.697266   1.000
37  광교1동  3794.630859   0.674
27   매산동  3974.218750   0.392
41  영통3동  3311.927734   0.389
16  권선1동  3312.462891   0.374
13    평동  2771.009766   0.310
36   원천동  2055.916016   0.301
31   행궁동  4634.782227   0.289
17   곡선동  3577.633789   0.273
34  매탄3동  2094.545410   0.255


In [42]:
df_sorted[['행정동', '적정 주차면수', '예측_적정주차면수', '주차면수 합', '필요주차면수', '유입인구', '우선순위지수']].head(10)

,행정동,적정 주차면수,예측_적정주차면수,주차면수 합,필요주차면수,유입인구,우선순위지수
25,인계동,22319.0312,13662.697266,7823,5839.697266,57072,1.000
37,광교1동,8112.4000,8947.630859,5153,3794.630859,59223,0.674
27,매산동,4748.2300,5532.218750,1558,3974.218750,32910,0.392
41,영통3동,8548.0600,9221.927734,5910,3311.927734,39170,0.389
16,권선1동,7467.5400,7399.462891,4087,3312.462891,37583,0.374
13,평동,8724.3500,8823.009766,6052,2771.009766,37331,0.310
36,원천동,6887.6600,7023.916016,4968,2055.916016,48780,0.301
31,행궁동,7313.4800,7715.782227,3081,4634.782227,20771,0.289
17,곡선동,11512.4500,10567.633789,6990,3577.633789,25418,0.273
34,매탄3동,6127.9000,6078.545410,3984,2094.545410,40571,0.255
